# Introduction to SPIFFE and SPIRE

SPIFFE and SPIRE (the SPIFFE Runtime Environment) are a set of APIs and associated tooling that provide a uniform language for describing service identity in a wide range of workloads (including orchestration systems), verifying that identity, and providing a workload with documents that serve as proof of that identity. The identities are in turn used for establishing trust between software systems across a wide variety of hosting platforms.

Concretely, SPIRE exposes the SPIFFE Workload API, which can authenticate and verify running software systems ("attestation" in SPIFFE-speak) and issue SPIFFE IDs and SVIDs to them. This in turn allows two workloads to establish trust between each other, for example by establishing an mTLS connection or by signing and verifying a JWT token. Or SPIFFE can be used to enable a workload to securely authenticate to a secret store, a database, or a cloud provider service.

This scenario provides a walk-through of installing the components that make up a SPIRE deployment and a basic demonstration of its functionality in action. During the exercise you will:

 * Install and start a SPIRE Server and SPIRE Agent
 * Attest a SPIRE Agent using join token node attestation
 * Create a workload registration entry
 * Simulate the workload API interaction and retrieve a workload SVID
 

# Step 1 of 3: Install, Configure and Run SPIRE Server

First, retrieve the latest SPIRE release to the machine you intend to perform the installation from with: `wget https://github.com/spiffe/spire/releases/download/v0.12.1/spire-0.12.1-linux-x86_64-glibc.tar.gz`

Once you have the file, proceed to extract/uncompress the tarball with: `tar -xvf spire-0.12.1-linux-x86_64-glibc.tar.gz -C .`

The tarball includes:

- The spire-agent and spire-server binaries
- Configuration files for the SPIRE Agent and Server

You can examine the files in the uncompressed directory: `ls -lrt ./spire-0.12.1`


In [ ]:
wget https://github.com/spiffe/spire/releases/download/v0.12.1/spire-0.12.1-linux-x86_64-glibc.tar.gz
tar -xvf spire-0.12.1-linux-x86_64-glibc.tar.gz -C .
ls -lrt .//spire-0.12.1

As we are about to install the server, you want to pay particular attention at the server configuration file: cat server.conf

In [ ]:
cat ./spire-0.12.1/conf/server/server.conf

Configure the trust_domain of the SPIRE Server in server.conf. The value corresponds to the host part of the authority component of the URI as defined in the SPIFFE standard: https://github.com/spiffe/spiffe/blob/master/standards/SPIFFE-ID.md#21-trust-domain

We will modify the following settings to cope with this workshop environment:

- trust_domain set to hpedev.io
- bind_port set to a unique id based on your student ID
- registration_uds_path set to use a local socket instead of a default system wide socket

>Note: The trust domain represents to the trust root of a system. A trust domain can be an individual, organization, environment or department running their own independent SPIFFE infrastructure. All workloads identified in the same trust domain are issued identity documents that can be verified against the root keys of the trust domain.

Make any other necessary modifications to the server file. For the purpose of this exercise we won't be making any other changes.

In [ ]:
export id={{ STDID }} 
export PORT=142$(echo "${id: -2}")
mkdir tmp
sed -i 's/example.org/hpedev.io/' ./spire-0.12.1/conf/server/server.conf
sed -i "s/8081/$PORT/" ./spire-0.12.1/conf/server/server.conf
sed -i 's/server {/server { \n    registration_uds_path = ".\/tmp\/spire-registration.sock"/' ./spire-0.12.1/conf/server/server.conf
cat ./spire-0.12.1/conf/server/server.conf

Next, we can check our config file with: `spire-server validate`, and follow the recommendation of a shorter TTL by editing the server.conf file (optional).

In [ ]:
./spire-0.12.1/bin/spire-server validate -config ./spire-0.12.1/conf/server/server.conf 

Next, we start SPIRE Server with: `spire-server run`

In [ ]:
./spire-0.12.1/bin/spire-server run -config ./spire-0.12.1/conf/server/server.conf 2>&1 >/dev/null &

We can verify that the SPIRE server is running using the healthcheck command: `spire-server healthcheck`

In [ ]:
./spire-0.12.1/bin/spire-server healthcheck -registrationUDSPath "./tmp/spire-registration.sock"

That concludes Step 1. 

# Step 2 of 3: Install, Configure and Run SPIRE Agent

Up next, you will perform the installation of the SPIRE Agent.

You will start by inspecting the default agent configuration file: `cat ./spire-0.12.1/conf/agent/agent.conf`

Next, you will set the trust domain to match the same trust domain as configured in the corrresponding SPIRE Server configuration: `hpedev.io`
We will also change the agent socket to be local to cope with this workshop environment.



In [ ]:
cat ./spire-0.12.1/conf/agent/agent.conf

In [ ]:
sed -i 's/example.org/hpedev.io/g' ./spire-0.12.1/conf/agent/agent.conf
sed -i "s/8081/$PORT/g" ./spire-0.12.1/conf/agent/agent.conf
sed -i "s/\/tmp\/agent.sock/.\/tmp\/agent.sock/g" ./spire-0.12.1/conf/agent/agent.conf
cat ./spire-0.12.1/conf/agent/agent.conf

There are several attestation methods by which an agent will be attested to join a trust domain. For the purpose of this exercise, we will use join token node attestation. The join_token plugin attests a node based on a pre-shared, one-time-use join token. A token must be generated by the server before it can be used to attest a node.

Conveniently, tokens may be generated through the CLI utility (`spire-server token generate`). You can generate a Join Token by running the command below on the SPIRE Server: `spire-server token generate -spiffeID spiffe://hpedev.io/clientnode`


In [ ]:
TOKEN=$(./spire-0.12.1/bin/spire-server token generate -registrationUDSPath "./tmp/spire-registration.sock" -spiffeID spiffe://hpedev.io/clientnode | awk '{print $2}' | tr -d '\r' )
echo $TOKEN

Now that a token has been generated, proceed to start the SPIRE Agent using the token generated earlier `spire-agent run -config agent.conf -joinToken $TOKEN &`

>Notes: The join_token is responsible for attesting the agent's identity using a one-time-use pre-shared key. The join token itself is configured by a CLI flag (-joinToken).


In [ ]:
./spire-0.12.1/bin/spire-agent run -config ./spire-0.12.1/conf/agent/agent.conf -joinToken $TOKEN 2>&1 >/dev/null  &

We can verify that the SPIRE agent is running, by running the healthcheck command with `spire-agent healthcheck`

In [ ]:
./spire-0.12.1/bin/spire-agent healthcheck -socketPath ./tmp/agent.sock 

That concludes Step 2. 

# Step 3 of 3: Register a workload and fetch its corresponding SVID

Now that both SPIRE Server and SPIRE Agent are up and running, you want to see how SPIRE works in action and check how workloads are identified.

The next step is to register a SPIFFE ID and a set of selectors that need to matched to obtain that SPIFFE ID. This is the SPIFFE ID of a workload that would start in this environment and retrieve its identity after being attested. To identify this workload, we will define selectors that will be mapped to a target SPIFFE ID. For the purpose of this exercise, the selectors use will be unix kernel based.

We will register the uid of the logged-in user as a selector of the workload's SPIFFE ID. During kernel based attestation the workload process will be interrogated for the registered uid.

Do create a worksload registration entry we use `spire-server entry create` with: 

- a parentID: this effectively tells SPIRE where a particular workload should be running and, by extension, which agents are authorized to ask for SVIDs on its behalf
- a spiffeID: when we see this workload, what SPIFFEID should we issue it? 
- and a selector: SPIRE needs some information that helps it to identify the workload, which is where the selectors discovered from attestation come in (the unix kernel selector in our case). 

Before we do this let's collect our Unix user ID (uid) which we will use to create an entry.


In [ ]:
uid=$(cat /etc/passwd | grep student{{ STDID }} | cut -f 3 -d:) 
echo $uid

In [ ]:
./spire-0.12.1/bin/spire-server entry create \
    -registrationUDSPath "./tmp/spire-registration.sock" \
    -parentID spiffe://hpedev.io/clientnode \
    -spiffeID spiffe://hpedev.io/workload \
    -selector unix:uid:$uid

We can list the list of entries already distributed by the server with `spire-server entry show`:

In [ ]:
./spire-0.12.1/bin/spire-server entry show -registrationUDSPath "./tmp/spire-registration.sock"

As a way to simulate the workload API interaction and retrieve the workload SVID bundle a workload would obtain, you can run the following api subcommand in the agent: `spire-agent api fetch x509`and examine the output.

In [ ]:
./spire-0.12.1/bin/spire-agent api fetch x509 --socketPath ./tmp/agent.sock

Optionally, you may write the SVID and key retrieved to disk with -write in order to examine in more detail:

- the SVID
- the key 
- the trust bundle

Then use the `openssl` to display the details of the issued certificate. 

In [ ]:
./spire-0.12.1/bin/spire-agent api fetch x509 --socketPath ./tmp/agent.sock -write .

In [ ]:
openssl x509 -in svid.0.pem -text -noout

This terminates our workshop on SPIFFE/SPIRE. Thank you for exploring the fundamentals of SPIFFE/SPIRE.


Jump to the [conclusion](2-WKSHP-Conclusion.ipynb).

